In [ ]:
import numpy as np
import pandas as pd
import os 
import torch.utils.data as data
import keras
import random 
import itertools
from keras import models
from keras import layers
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras.preprocessing.image import img_to_array,array_to_img, load_img
from keras.utils import to_categorical
from keras import backend as K
from keras import metrics
from keras.optimizers import Adam, RMSprop,SGD
from keras.preprocessing import image
# from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
# from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from numpy import array
from PIL import Image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import cv2

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
base_skin_dir = os.path.join('', 'dataset')

import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
# Graphics in SVG| format are more sharp and legible
%config InlineBackend.figure_format = 'svg'



In [ ]:
IMAGE_DIMS = (100, 75, 3)
# default loading
def load_img(imagePath):
    image = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    return image

# return image with higher increase brightness
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = cv2.resize(hsv, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img_to_array(img)
    return img

# return thresholding featrue transformation
def thresholding_feature(imagePath):
    img_grey = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
    img_grey = cv2.resize(img_grey, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    # # Adaptive Gaussian
    img_grey = cv2.adaptiveThreshold(img_grey, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(img_grey, (3, 3), 0)
    ret3, img_binary = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # invert black = 255
    # ret, thresh1 = cv2.threshold(img_binary, 157, 255, cv2.THRESH_BINARY_INV)
    img_binary = img_to_array(img_binary)
    return img_binary 

def load_atm_img(imagePath):
    img = cv2.imread(imagePath,0)
    img = cv2.resize(img, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    atm_img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
    # TODO make it gray     
    atm_img = img_to_array(atm_img)
    return atm_img

def load_img_with_cv2(imagePath, frth_channel = 'without'):
    channel = False
    image = load_img(imagePath)
    sub_channel = 'no_channel'
    if frth_channel == 'atm':
        sub_channel = load_atm_img(imagePath)
        channel = True
    elif frth_channel == 'gray': 
        sub_channel = increase_brightness(image, 150)
        channel = True
    elif frth_channel == 'threshold':
        sub_channel = thresholding_feature(imagePath)
        channel = True
    image = img_to_array(image)
    if channel == True:
        multiImage = np.concatenate((image, sub_channel), axis=2) 
        return multiImage
    else: 
        return image
    
def load_gray(imagePath):
    image = load_img(imagePath)
    return increase_brightness(image, 150)

# Image loading

In [ ]:
data = []
for img_path in current_skin_data['path']:
    data.append(load_img_with_cv2(img_path))
labels = np.array(current_skin_data['dx'])

# Image Normalization

In [ ]:
labels = np.array(labels)
data = np.array(data)

generate_labels = np.array(generate_labels)
full_labels = np.concatenate((labels, generate_labels))

generate_data = np.array(generate_data, dtype=np.uint8)
full_data = np.concatenate((data, generate_data))

full_data_mean = np.mean(full_data)
full_data_std = np.std(full_data)
full_data = (full_data - full_data_mean)/full_data_std

mlb = LabelBinarizer()
full_labels = mlb.fit_transform(full_labels)

data_train, x_test, data_test, y_test = train_test_split(
        full_data, full_labels, test_size=0.2, random_state=42)

x_train, x_validate, y_train, y_validate = train_test_split(
        data_train, data_test, 
        test_size = 0.1, random_state = 2)

# Define metrics and optimizer for our model

In [ ]:
def f1(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
def task_optimizer(optimizer):
    if (optimizer == "SGD"):
        return SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    elif (optimizer == "RMSprop"):
        return RMSprop()
    elif (optimizer == "Adam"):
        return Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    else: 
        return Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

OPT = task_optimizer('RMSprop')
OPT

In [ ]:
from keras.metrics import categorical_accuracy
from keras import backend as K
K.set_image_dim_ordering('tf')

# Define model function

In [ ]:
def get_model(output_neurons = 7): 
    #  input shape using shape off test samples
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=(100,75,3)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3)))
    model.add(layers.Conv2D(128, (3, 3)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(256))
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.1))

    model.add(layers.Dense(256))
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.1))

    model.add(layers.Dense(output_neurons))
    model.add(layers.Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer = OPT,
                  metrics=['categorical_accuracy'])
    model.summary()
    return model

# Prepare model for training 

In [ ]:
# model = get_model()
labels = np.array(labels)
data = np.array(data)
generate_labels = np.array(generate_labels)
generate_data = np.array(generate_data, dtype=np.uint8)

full_labels = np.concatenate((labels, generate_labels))
full_data = np.concatenate((data, generate_data))

full_data_mean = np.mean(full_data)
full_data_std = np.std(full_data)
full_data = (full_data - full_data_mean)/full_data_std
print('Данные нормализованы')
mlb = LabelBinarizer()
full_labels = mlb.fit_transform(full_labels)

data_train, x_test, data_test, y_test = train_test_split(
    full_data, full_labels, test_size=0.2, random_state=42)
    
x_train, x_validate, y_train, y_validate = train_test_split(
    data_train, data_test, 
    test_size = 0.1, random_state = 2)
print('Данные разбиты на тестовые, валидационные и тренировочные')
MD = get_model()
history = MD.fit(x_train, y_train, epochs=20, batch_size=64 , validation_data=(x_validate, y_validate))
loss_test, accuracy_test = MD.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = MD.evaluate(x_validate, y_validate, verbose=1)
loss_train, accuracy_train = MD.evaluate(x_train, y_train, verbose=1)

# Show Results 

In [ ]:
loss, accuracy = MD.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = MD.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss = %f  "  % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f ;" % (accuracy, loss))
# model.save_weights("weights.h5")
# print("Saved model to disk")

# Show graphic results

In [ ]:
# Function to plot confusion matrix    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = MD.predict(x_validate)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_validate,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(7)) 